In [1]:
import os
import fitz
import pandas as pd
from pathlib import Path

In [2]:
#To iterate over all files in the data set
#call the folder containing world bank loan agreements

directory  = "dataWBloans"

In [55]:
#create a list of the file names in the directory
#.pdf files only

file_name=[]

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        file_name.append(filename)
        continue
    else:
        continue

In [56]:
#each element of the list is a string.
#for example, file_name[0] ='1995_march_ ...conformed.pdf' and, 
#type(file_name[0]) = str

In [57]:
#create a list of the date of the agreements
#extract the dates from the first part of the file names.

date = []
for i in range(len(file_name)):
    index = []
    for j in range(len(file_name[i])):
        if file_name[i][j] == '_':
            index.append(j)
    date.append(file_name[i][:index[2]].replace("_",""))

#date = ['1995march23','1995july7','1997december19',...]

In [60]:
#replace 'month' by the corresponding number

for i in range(len(date)):
        date[i] = date[i].replace("january","-1-")
        date[i] = date[i].replace("february","-2-")
        date[i] = date[i].replace("march","-3-")
        date[i] = date[i].replace("april","-4-")
        date[i] = date[i].replace("may","-5-")
        date[i] = date[i].replace("june","-6-")
        date[i] = date[i].replace("july","-7-")
        date[i] = date[i].replace("august","-8-")
        date[i] = date[i].replace("september","-9-")
        date[i] = date[i].replace("october","-10-")
        date[i] = date[i].replace("november","-11-")
        date[i] = date[i].replace("december","-12-")
        
#date = ['1995-3-23','1995-7-7','1997-12-19',...]

In [61]:
#I don't understand how this code works... 
files = Path(directory).glob('*')

#create a list of the text of cover pages of the documents
cover = []
for file in files:
    text = ""
    doc = fitz.open(file)
    page_0 = doc.load_page(0)
    text += page_0.getText()
    cover.append(text)

In [62]:
project = []

#extract a string inside of the fist parenthesis
for text in cover:
    for i in range(len(text)): 
        if text[i] == '(':
            openup = i
            break
    for j in range(len(text)):
        if text[j] == ')':
            closedown = j
            break
    project.append(text[openup+1: closedown])

In [74]:
loan_number = []
borrower = []
#look for a substring 'LOAN' and 'NUMBER' and
#extract the code next to 'LOAN NUMBER'

for text in cover:
    wordlist = text.split()
    dummy_loan = 0
    for word in wordlist:
        index = wordlist.index(word)
        if 'LOAN' in wordlist and wordlist.index(word) ==  wordlist.index('LOAN') + 1:
            loan_number.append(wordlist[index+1] + wordlist[index+2])
            dummy_loan += 1
            break
    if dummy_loan == 0:
        loan_number.append('N/A')
        
    dummy_borrower = 0
    for word in wordlist:
        index = wordlist.index(word)
        if 'and' in wordlist and 'between' in wordlist and wordlist.index(word) ==  wordlist.index('between') + 1:
            index_and = wordlist.index('and')
            borrower_name = ''
            for country in range(index,index_and):
                    borrower_name += wordlist[country]+' '
            borrower.append(borrower_name)
            dummy_borrower += 1    
            break
    if dummy_borrower == 0:
        borrower.append('N/A')
    

In [75]:
borrower

['N/A',
 'RUSSIAN FEDERATION ',
 'Former Yugoslav Republic of Macedonia (the Borrower) ',
 'UKRAINE ',
 'REPUBLIC OF CAMEROON ',
 'INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT ',
 'REPUBLIC OF SERBIA ',
 'REPUBLIC OF INDONESIA ',
 'INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT ',
 'INDIA ',
 '',
 'N/A',
 'INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT ',
 '',
 'ROMANIA ',
 'INDIA ',
 'REPUBLIC OF INDONESIA ',
 'KINGDOM OF MOROCCO ',
 'ILLER BANKASI ANONIM SIRKETI ',
 '',
 'N/A',
 'REPUBLIC OF INDONESIA ',
 'PEOPLE’S REPUBLIC OF CHINA ',
 'N/A',
 'INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT ',
 "PEOPLE'S REPUBLIC OF CHINA ",
 'REPUBLIC OF COSTA RICA ',
 'ISLAMIC REPUBLIC OF PAKISTAN ',
 'REPUBLIC OF PERU ',
 'UNITED MEXICAN STATES ',
 '',
 '',
 'N/A',
 'UNITED MEXICAN STATES ',
 'GRENADA ',
 '',
 'INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT ',
 'KINGDOM OF MOROCCO ',
 '',
 '',
 'N/A',
 'INDIA ',
 'PEOPLE’S DEMOCRATIC REPUBLIC OF ALGERIA ',
 ''

In [76]:
dictionary = {'loan_number':loan_number, 'borrower':borrower, 'date': date, 'project':project}
df = pd.DataFrame(data = dictionary)
df

,loan_number,borrower,date,project
0,N/A,N/A,1995-3-23,
1,3898RU,RUSSIAN FEDERATION,1995-7-7,Second Rehabilitation Loan
2,4240MK,Former Yugoslav Republic of Macedonia (the Bor...,1997-12-19,Second Private Sector Development Project
3,8404-UALoan,UKRAINE,2014-7-9,Social Safety Nets Modernization Project
4,3206CM,REPUBLIC OF CAMEROON,1990-8-3,Social Dimensions of Adjustment Project
...,...,...,...,...
3200,79830-RULoan,,2011-3-14,Financial Education and Financial Literacy Pro...
3201,3169BR,,1991-4-30,Highways Management and Rehabilitation Project
3202,N/A,N/A,1994-2-3,
3203,N/A,N/A,1995-5-9,Disclosure Authorized\nPublic Disclosure Author


In [81]:
df.to_csv('0512_lbdp.csv')